# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 12)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 12:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


Moe_Szyslak: Ah, isn't that nice. Now, there is a politician

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counts = Counter(text)
    s_vocab = sorted(counts, key=counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(s_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [7]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    
    token = {}
    token['.'] = '||PERIOD||'
    token[','] = '||COMMA||'
    token['"'] = '||QUOTATION_MARK||'
    token[';'] = '||SEMICOLON||'
    token['!'] = '||EXCLAMATION_MARK||'
    token['?'] = '||QUESTION_MARK||'
    token['('] = '||LEFT_PAREN||'
    token[')'] = '||RIGHT_PAREN||'
    token['--'] = '||HYPHENS||'
    token['\n'] = '||RETURN||'
    
    return token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.9.0
Default GPU Device: /device:GPU:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [3]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    target = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return (inputs, target, learning_rate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [113]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    
    keep_prob = 0.7
    layers = 4
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    cell = tf.contrib.rnn.MultiRNNCell([drop for _ in range(layers)])
    
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name="initial_state")
    
    return (cell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [7]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [9]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    output, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)    
    final_state = tf.identity(final_state, name="final_state")
    
    return (output, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [119]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    emb_out = get_embed(input_data, vocab_size, rnn_size)
    rnn_out, rnn_est = build_rnn(cell, emb_out)
    
    predictions = tf.contrib.layers.fully_connected(rnn_out, vocab_size, activation_fn=None)
    
    return predictions, rnn_est


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [95]:
import math
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    batch_counter = int(len(int_text) / (batch_size * seq_length))
    word_length = batch_size * seq_length
    
    int_text = int_text[:batch_counter*word_length]

    a = np.array(int_text)
    a_batch = np.split(a.reshape(batch_size, -1), batch_counter, 1)
    
    b = np.array(int_text[1:] + [int_text[0]])
    b_batch = np.split(b.reshape(batch_size, -1), batch_counter, 1)
    
    z_batches = list(zip(a_batch, b_batch))
    
    return np.array(z_batches)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [126]:
# Number of Epochs
num_epochs = 300
# Batch Size
batch_size = 80
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [127]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/57   train_loss = 8.822
Epoch   0 Batch   10/57   train_loss = 6.746
Epoch   0 Batch   20/57   train_loss = 6.470
Epoch   0 Batch   30/57   train_loss = 6.205
Epoch   0 Batch   40/57   train_loss = 6.263
Epoch   0 Batch   50/57   train_loss = 6.170
Epoch   1 Batch    3/57   train_loss = 6.124
Epoch   1 Batch   13/57   train_loss = 6.009
Epoch   1 Batch   23/57   train_loss = 5.963
Epoch   1 Batch   33/57   train_loss = 6.055
Epoch   1 Batch   43/57   train_loss = 6.003
Epoch   1 Batch   53/57   train_loss = 6.013
Epoch   2 Batch    6/57   train_loss = 6.065
Epoch   2 Batch   16/57   train_loss = 5.984
Epoch   2 Batch   26/57   train_loss = 5.994
Epoch   2 Batch   36/57   train_loss = 5.917
Epoch   2 Batch   46/57   train_loss = 6.071
Epoch   2 Batch   56/57   train_loss = 6.145
Epoch   3 Batch    9/57   train_loss = 5.924
Epoch   3 Batch   19/57   train_loss = 5.896
Epoch   3 Batch   29/57   train_loss = 5.834
Epoch   3 Batch   39/57   train_loss = 5.746
Epoch   3 

Epoch  32 Batch    6/57   train_loss = 3.571
Epoch  32 Batch   16/57   train_loss = 3.564
Epoch  32 Batch   26/57   train_loss = 3.552
Epoch  32 Batch   36/57   train_loss = 3.453
Epoch  32 Batch   46/57   train_loss = 3.562
Epoch  32 Batch   56/57   train_loss = 3.611
Epoch  33 Batch    9/57   train_loss = 3.498
Epoch  33 Batch   19/57   train_loss = 3.434
Epoch  33 Batch   29/57   train_loss = 3.376
Epoch  33 Batch   39/57   train_loss = 3.498
Epoch  33 Batch   49/57   train_loss = 3.494
Epoch  34 Batch    2/57   train_loss = 3.409
Epoch  34 Batch   12/57   train_loss = 3.586
Epoch  34 Batch   22/57   train_loss = 3.413
Epoch  34 Batch   32/57   train_loss = 3.367
Epoch  34 Batch   42/57   train_loss = 3.349
Epoch  34 Batch   52/57   train_loss = 3.310
Epoch  35 Batch    5/57   train_loss = 3.343
Epoch  35 Batch   15/57   train_loss = 3.491
Epoch  35 Batch   25/57   train_loss = 3.188
Epoch  35 Batch   35/57   train_loss = 3.301
Epoch  35 Batch   45/57   train_loss = 3.355
Epoch  35 

Epoch  64 Batch   12/57   train_loss = 2.627
Epoch  64 Batch   22/57   train_loss = 2.563
Epoch  64 Batch   32/57   train_loss = 2.627
Epoch  64 Batch   42/57   train_loss = 2.516
Epoch  64 Batch   52/57   train_loss = 2.521
Epoch  65 Batch    5/57   train_loss = 2.525
Epoch  65 Batch   15/57   train_loss = 2.632
Epoch  65 Batch   25/57   train_loss = 2.464
Epoch  65 Batch   35/57   train_loss = 2.565
Epoch  65 Batch   45/57   train_loss = 2.580
Epoch  65 Batch   55/57   train_loss = 2.643
Epoch  66 Batch    8/57   train_loss = 2.590
Epoch  66 Batch   18/57   train_loss = 2.601
Epoch  66 Batch   28/57   train_loss = 2.506
Epoch  66 Batch   38/57   train_loss = 2.635
Epoch  66 Batch   48/57   train_loss = 2.611
Epoch  67 Batch    1/57   train_loss = 2.611
Epoch  67 Batch   11/57   train_loss = 2.626
Epoch  67 Batch   21/57   train_loss = 2.628
Epoch  67 Batch   31/57   train_loss = 2.630
Epoch  67 Batch   41/57   train_loss = 2.635
Epoch  67 Batch   51/57   train_loss = 2.579
Epoch  68 

Epoch  96 Batch   18/57   train_loss = 2.008
Epoch  96 Batch   28/57   train_loss = 1.980
Epoch  96 Batch   38/57   train_loss = 2.085
Epoch  96 Batch   48/57   train_loss = 1.989
Epoch  97 Batch    1/57   train_loss = 2.019
Epoch  97 Batch   11/57   train_loss = 2.032
Epoch  97 Batch   21/57   train_loss = 2.121
Epoch  97 Batch   31/57   train_loss = 2.062
Epoch  97 Batch   41/57   train_loss = 2.018
Epoch  97 Batch   51/57   train_loss = 1.982
Epoch  98 Batch    4/57   train_loss = 1.927
Epoch  98 Batch   14/57   train_loss = 1.969
Epoch  98 Batch   24/57   train_loss = 1.927
Epoch  98 Batch   34/57   train_loss = 1.969
Epoch  98 Batch   44/57   train_loss = 1.979
Epoch  98 Batch   54/57   train_loss = 2.026
Epoch  99 Batch    7/57   train_loss = 1.935
Epoch  99 Batch   17/57   train_loss = 1.976
Epoch  99 Batch   27/57   train_loss = 1.928
Epoch  99 Batch   37/57   train_loss = 1.916
Epoch  99 Batch   47/57   train_loss = 1.979
Epoch 100 Batch    0/57   train_loss = 1.926
Epoch 100 

Epoch 128 Batch   24/57   train_loss = 1.604
Epoch 128 Batch   34/57   train_loss = 1.612
Epoch 128 Batch   44/57   train_loss = 1.600
Epoch 128 Batch   54/57   train_loss = 1.635
Epoch 129 Batch    7/57   train_loss = 1.588
Epoch 129 Batch   17/57   train_loss = 1.642
Epoch 129 Batch   27/57   train_loss = 1.633
Epoch 129 Batch   37/57   train_loss = 1.561
Epoch 129 Batch   47/57   train_loss = 1.646
Epoch 130 Batch    0/57   train_loss = 1.581
Epoch 130 Batch   10/57   train_loss = 1.574
Epoch 130 Batch   20/57   train_loss = 1.639
Epoch 130 Batch   30/57   train_loss = 1.675
Epoch 130 Batch   40/57   train_loss = 1.676
Epoch 130 Batch   50/57   train_loss = 1.484
Epoch 131 Batch    3/57   train_loss = 1.654
Epoch 131 Batch   13/57   train_loss = 1.568
Epoch 131 Batch   23/57   train_loss = 1.609
Epoch 131 Batch   33/57   train_loss = 1.617
Epoch 131 Batch   43/57   train_loss = 1.635
Epoch 131 Batch   53/57   train_loss = 1.587
Epoch 132 Batch    6/57   train_loss = 1.616
Epoch 132 

Epoch 160 Batch   30/57   train_loss = 1.458
Epoch 160 Batch   40/57   train_loss = 1.420
Epoch 160 Batch   50/57   train_loss = 1.312
Epoch 161 Batch    3/57   train_loss = 1.401
Epoch 161 Batch   13/57   train_loss = 1.362
Epoch 161 Batch   23/57   train_loss = 1.411
Epoch 161 Batch   33/57   train_loss = 1.391
Epoch 161 Batch   43/57   train_loss = 1.492
Epoch 161 Batch   53/57   train_loss = 1.484
Epoch 162 Batch    6/57   train_loss = 1.335
Epoch 162 Batch   16/57   train_loss = 1.407
Epoch 162 Batch   26/57   train_loss = 1.414
Epoch 162 Batch   36/57   train_loss = 1.324
Epoch 162 Batch   46/57   train_loss = 1.377
Epoch 162 Batch   56/57   train_loss = 1.474
Epoch 163 Batch    9/57   train_loss = 1.444
Epoch 163 Batch   19/57   train_loss = 1.346
Epoch 163 Batch   29/57   train_loss = 1.370
Epoch 163 Batch   39/57   train_loss = 1.353
Epoch 163 Batch   49/57   train_loss = 1.344
Epoch 164 Batch    2/57   train_loss = 1.413
Epoch 164 Batch   12/57   train_loss = 1.453
Epoch 164 

Epoch 192 Batch   36/57   train_loss = 1.142
Epoch 192 Batch   46/57   train_loss = 1.118
Epoch 192 Batch   56/57   train_loss = 1.152
Epoch 193 Batch    9/57   train_loss = 1.175
Epoch 193 Batch   19/57   train_loss = 1.105
Epoch 193 Batch   29/57   train_loss = 1.090
Epoch 193 Batch   39/57   train_loss = 1.121
Epoch 193 Batch   49/57   train_loss = 1.087
Epoch 194 Batch    2/57   train_loss = 1.151
Epoch 194 Batch   12/57   train_loss = 1.136
Epoch 194 Batch   22/57   train_loss = 1.089
Epoch 194 Batch   32/57   train_loss = 1.148
Epoch 194 Batch   42/57   train_loss = 1.134
Epoch 194 Batch   52/57   train_loss = 1.100
Epoch 195 Batch    5/57   train_loss = 1.079
Epoch 195 Batch   15/57   train_loss = 1.199
Epoch 195 Batch   25/57   train_loss = 1.140
Epoch 195 Batch   35/57   train_loss = 1.141
Epoch 195 Batch   45/57   train_loss = 1.215
Epoch 195 Batch   55/57   train_loss = 1.105
Epoch 196 Batch    8/57   train_loss = 1.115
Epoch 196 Batch   18/57   train_loss = 1.159
Epoch 196 

Epoch 224 Batch   42/57   train_loss = 0.934
Epoch 224 Batch   52/57   train_loss = 1.050
Epoch 225 Batch    5/57   train_loss = 0.931
Epoch 225 Batch   15/57   train_loss = 1.072
Epoch 225 Batch   25/57   train_loss = 0.961
Epoch 225 Batch   35/57   train_loss = 1.013
Epoch 225 Batch   45/57   train_loss = 0.988
Epoch 225 Batch   55/57   train_loss = 0.997
Epoch 226 Batch    8/57   train_loss = 1.020
Epoch 226 Batch   18/57   train_loss = 0.989
Epoch 226 Batch   28/57   train_loss = 0.949
Epoch 226 Batch   38/57   train_loss = 1.014
Epoch 226 Batch   48/57   train_loss = 1.019
Epoch 227 Batch    1/57   train_loss = 1.049
Epoch 227 Batch   11/57   train_loss = 0.941
Epoch 227 Batch   21/57   train_loss = 1.078
Epoch 227 Batch   31/57   train_loss = 0.982
Epoch 227 Batch   41/57   train_loss = 1.013
Epoch 227 Batch   51/57   train_loss = 1.019
Epoch 228 Batch    4/57   train_loss = 0.846
Epoch 228 Batch   14/57   train_loss = 0.942
Epoch 228 Batch   24/57   train_loss = 0.899
Epoch 228 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [2]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [4]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    
    # I'look for help in the internet to do this part, I couldn't figure out by myself. 
    
    # TODO: Implement Function
    word = np.random.choice(len(int_to_vocab), p=probabilities)
    return int_to_vocab[word]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [6]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: who'da thought such a new stuff instead of a big dumb valley and clear in me, carl. if i'm interested here one at the family-owned door!
moe_szyslak:(into phone) who sure about to hoping you money for me!
moe_szyslak: no. get outside, edna after my friends call it can't buy jesus as her drive, and i'm gonna see this moolah-stealing homer!
barney_gumble: i did why i want the three more in our cash!
moe_szyslak: i'm very little he too thing, ladies. let's think so is our takeaway.
moe_szyslak:(" gun) the drunk forget-me-shot. clear as you many man has caught one and market.(sips") dr. hibbert szyslak, hi and somethin'. no gets good time everything
whoa what if i wanted your wife directions for a other man, ah, act are the power broad for me?
moe_szyslak:(offended)" too thing.
ned_flanders: c..(short the face) whoa.(chuckles)
marge_simpson: sure.
lisa_simpson: i knew


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.